In [1]:
!rm -fr r_trader out
!mkdir out
!git clone https://github.com/abreham-atlaw/r_trader
!cd r_trader &&  git checkout deep-reinforcement.training-experiment-linear
!pip install cattrs positional-encodings==6.0.1 dropbox==11.27.0 pymongo==4.3.3 dependency-injector==4.41.0

Cloning into 'r_trader'...
remote: Enumerating objects: 12930, done.
remote: Counting objects: 100% (3844/3844), done.
remote: Compressing objects: 100% (1073/1073), done.
remote: Total 12930 (delta 2830), reused 3767 (delta 2753), pack-reused 9086 (from 1)
Receiving objects: 100% (12930/12930), 76.53 MiB | 11.92 MiB/s, done.
Resolving deltas: 100% (9162/9162), done.
Branch 'deep-reinforcement.training-experiment-linear' set up to track remote branch 'deep-reinforcement.training-experiment-linear' from 'origin'.
Switched to a new branch 'deep-reinforcement.training-experiment-linear'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 582.5/582.5 kB 14.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 69.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 14.9 MB/s eta 0:00:00


In [2]:
import sys
sys.path.append("/kaggle/working/r_trader")

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam, SGD, Adagrad
import matplotlib.pyplot as plt

import os
import signal 

from core.utils.research.data.load.dataset import BaseDataset
from core.utils.research.training.trainer import Trainer
from core.utils.research.model.model.cnn.model import CNN
from core.utils.research.model.model.linear.model import LinearModel
from lib.utils.torch_utils.model_handler import ModelHandler
from core.utils.research.training.callbacks.checkpoint_callback import CheckpointCallback, StoreCheckpointCallback
from core.utils.research.training.data.repositories.checkpoint_repository import CheckpointRepository
from lib.utils.file_storage import PCloudClient
from core.utils.research.training.data.state import TrainingState
from core import Config
from core.utils.research.training.callbacks.metric_callback import MetricCallback
from core.utils.research.training.data.repositories.metric_repository import MetricRepository, MongoDBMetricRepository
from core.utils.kaggle import FusedManager
from core.di import init_di, ApplicationContainer
from core.utils.research.training.data.metric import MetricsContainer
from core.utils.research.model.layers import Indicators
from core.di import ServiceProvider
from core.utils.research.losses import OutputBatchVarianceLoss, MultiLoss
from core.utils.kaggle.data_repository import KaggleDataRepository

In [4]:
KAGGLE_ENV = os.path.exists("/kaggle/input")

DATA_ROOT = "/kaggle/input" if KAGGLE_ENV else "/content/input"

DATASETS = [
    f"abrehamatlaw0/spinoza-ds-datapreparer-simsim-cum-0-it-2-{i}"
    for i in range(0,4)
]
CHECKSUMS = None
KERNEL_MODE = False
ZIP_FILENAME = "out.zip"
if not KAGGLE_ENV:
    download_data(DATA_ROOT, DATASETS, ZIP_FILENAME, kernel_mode=KERNEL_MODE, checksums=CHECKSUMS)


CONTAINERS = [os.path.join(DATA_ROOT, container) for container in os.listdir(DATA_ROOT)]
DATA_PATHES, TEST_DATA_PATHES = [
    [
        os.path.join(container, "out", type_)
        for container in CONTAINERS
    ]
    for type_ in ["train", "test"]
]

NOTEBOOK_ID = "abrehamalemu/rtrader-training-exp-0-linear-124-cum-0-it-4-tot"
MODEL_ID = NOTEBOOK_ID.replace("/", "-")

NUM_FILES = None
DATA_CACHE_SIZE = 2
DATALOADER_WORKERS = 4

VOCAB_SIZE = 431
DROPOUT = 0.3
LAYER_SIZES = [512 for _ in range(5)] + [VOCAB_SIZE + 1]
HIDDEN_ACTIVATION = nn.LeakyReLU()
INIT_FUNCTION = None
NORM = [True] + [False for _ in LAYER_SIZES[1:]]
BLOCK_SIZE = 1148
LR = 1e-4
LOSS_WEIGHTS = [0.7, 0.3]

BATCH_SIZE = 64
EPOCHS = 300
TIMEOUT = 10*60*60

DTYPE = torch.float32
NP_DTYPE = np.float32

MODEL_URL = None
SAVE_PATH = os.path.abspath("./out/model.zip")
STATE_SAVE_PATH = os.path.abspath("./out/model.zip")

METRIC_REPOSITORY = MongoDBMetricRepository(
    Config.MONGODB_URL,
    MODEL_ID
)

CALLBACKS = [
    StoreCheckpointCallback(path=SAVE_PATH),
    MetricCallback(
       METRIC_REPOSITORY
    )
]

In [5]:
repository = CheckpointRepository(
    ServiceProvider.provide_file_storage()
)

In [6]:
state_model = repository.get(MODEL_ID)
# state_model = None
if state_model is None:
    print("[+]Creating a new model...")
    
    model = LinearModel(
        dropout_rate=DROPOUT,
        layer_sizes=LAYER_SIZES,
        hidden_activation=HIDDEN_ACTIVATION,
        init_fn=INIT_FUNCTION,
        norm=NORM,
        input_size=BLOCK_SIZE
    )
    
else:
    print("[+]Using loaded model...")
    state, model = state_model
state = TrainingState(
    epoch=0,
    batch=0,
    id=MODEL_ID
)

[+]Creating a new model...


In [7]:
dataset = BaseDataset(
    root_dirs=DATA_PATHES,
    out_dtypes=NP_DTYPE,
    num_files=NUM_FILES
)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, num_workers=DATALOADER_WORKERS, pin_memory=True)

In [8]:
test_dataset = BaseDataset(
    root_dirs=TEST_DATA_PATHES, 
    out_dtypes=NP_DTYPE,
    num_files=NUM_FILES
)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, num_workers=DATALOADER_WORKERS, pin_memory=True)

In [9]:
trainer = Trainer(model, callbacks=CALLBACKS)

In [10]:
trainer.cls_loss_function = MultiLoss(
    losses = [
        nn.CrossEntropyLoss(),
        OutputBatchVarianceLoss()
    ],
    weights = LOSS_WEIGHTS,
    device=trainer.device
)
trainer.reg_loss_function = nn.MSELoss()
trainer.optimizer = Adam(trainer.model.parameters(), lr=LR)

In [11]:
class TimeoutException(Exception):
    pass

def handle_timeout(*args, **kwargs):
    raise TimeoutException()

signal.signal(signal.SIGALRM, handle_timeout)
signal.alarm(TIMEOUT)

0

In [ ]:
try:
    trainer.train(dataloader, val_dataloader=test_dataloader, epochs=EPOCHS, progress=True, progress_interval=100, state=state, cls_loss_only=False)
except TimeoutException:
    pass

Model Summary
Layer Name							Number of Parameters
layers.0.weight			587776
layers.0.bias			512
layers.1.weight			262144
layers.1.bias			512
layers.2.weight			262144
layers.2.bias			512
layers.3.weight			262144
layers.3.bias			512
layers.4.weight			262144
layers.4.bias			512
layers.5.weight			221184
layers.5.bias			432
norms.0.weight			1148
norms.0.bias			1148
Total Params:1862824
[+]Shuffling dataset...


  0%|          | 0/32310 [00:00<?, ?it/s]/opt/conda/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 1 loss: 43.71457290649414(cls: 43.605010986328125, reg: 0.10955387353897095):   1%|          | 173/32310 [00:04<12:40, 42.23it/s]

In [ ]:
ModelHandler.save(model, SAVE_PATH)

In [ ]:
repository.update(trainer.state, trainer.model)

In [ ]:
metrics = MetricsContainer()
for metric in METRIC_REPOSITORY.get_all():
    metrics.add_metric(metric)

for i in range(3):
    train_losses = [metric.value[i] for metric in metrics.filter_metrics(source=0)]
    val_losses = [metric.value[i] for metric in metrics.filter_metrics(source=1)]
    plt.figure()
    plt.plot(train_losses)
    plt.plot(val_losses)
    plt.show()

In [ ]:
for X, y in test_dataloader:
    break
y_hat = model(X.to(trainer.device)).detach().cpu().numpy()

import matplotlib.pyplot as plt
def softmax(x):
    exp_x = np.exp(x - np.max(x))
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

def scale(x):
    x = softmax(x)
    x = x / np.max(x)
    return x

for i in range(y_hat.shape[0]):
    plt.figure()
    plt.plot(y[i, :-1])
    plt.plot(scale(y_hat[i, :-1]))
    

In [ ]:
!rm -fr r_trader